In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import csv
import numpy as np
from scipy import fft
from numpy import exp,sin,cos,pi
from numpy.linalg import norm
from pyquaternion import Quaternion 
from scipy.optimize import minimize

from utils import *

In [3]:
#Read in the raw data
raw_dat=load_endaq_log("data/accel_cal_")

#Pull out the magnetometer data in numpy form
dfMag=raw_dat['mag']
mag=dfMag.to_numpy()
# Get the magnetometer time series in seconds
ts=dfMag.index.to_numpy(dtype=np.float32)/1e9
ts=ts-ts[0]

# Synch the gyro to the magnetometer, convert to radians/s
gyro=(pi/180)*synchronize_series(raw_dat['gyro'],ref=dfMag).to_numpy()

#Sync the accelerometer to the gyro
#the 8g accelerometer is by far the best.
acc=   synchronize_series(raw_dat['acc8'],ref=dfMag).to_numpy()


In [4]:
##Apply an iteration of the optomization loop

   

def msqError(params,intervals,acc,gyro,mag,ts,g=np.array([0,0,9.799])):
    """
    Evaluate the msq error of the calibration/orientation model 
    params=[scalex, scaley,scalez, biasx,biasy,biasz, q_w,q_i,q_j,q_k]
    
    intervals=[(a1,b1),(a2,b2),...]
        sections of the calibration data set which are quiet
        used for calculating the msq error at the end
    
    acc:
        acclerometer time series
        
    gyro:
        gyroscope time series
    
    mag:
        magnetometer time series
    
    ts:
        time values
        
    make sure that acc,gyro,mag are time synced
    
    """
    scale=params[0:3]
    bias=params[3:6]
    q0=params[6:]
    
    ## Apply scale-bias calibration
    acc_calibrated=acc.dot(np.diag(scale))+bias
    
    #Apply the madgwick filter to get the heading correction
    #Using the initial orientation
    ACC_LAB,QIMU=apply_ahrs(gyro,acc_calibrated,mag,ts,q0=q0,g=g)
    
    # Use compare the measured acceleration in the lab frame to the gravitational model
    # When everything is calibrated, the lab frame acceleration should be zero during the time period
    # specified in intervals
    t_select,ACC_LAB_SELECT=idx_filter(ts,ACC_LAB,intervals)
    return 3*np.mean(ACC_LAB_SELECT**2)

In [ ]:
params=np.array([\
    1.0,1.0,1.0,\
     0.0,0.0,0.0,\
     1.0,0.0,0.0,0.0 \
    ])

intervals=[\
          (0,9),\
          (11,20),\
          (23,33),\
          (36.5,45.5),\
          (47,55),\
          (59,70),\
          ]       
print(msqError(params,intervals,acc,gyro,mag,ts))

func = lambda params: msqError(params,intervals,acc,gyro,mag,ts)
minimize(func,params)

0.0771140219246814


In [ ]:

## Downsample the accerometer measurements
dfIMU=raw_dat['accIMU'].resample('30ms').apply(np.mean)
df8=raw_dat['acc8'].resample('30ms').apply(np.mean)
df16=raw_dat['acc16'].resample('30ms').apply(np.mean)


## inject the IMU sensor timestamps and interpolate
df8 = df8.reindex(df8.index.union(dfIMU.index)).interpolate()
df16=df16.reindex(df16.index.union(dfIMU.index)).interpolate()

## Slice out only the synchronous estimates
df8=df8.loc[dfIMU.index]
df16=df16.loc[dfIMU.index]

## Reshape into matrix form for calculations
ts=df8.index.to_numpy(dtype=np.float32)/1e9
mat8=df8.T.to_numpy()
mat16=df16.T.to_numpy()
matIMU=dfIMU.T.to_numpy()


In [ ]:
## All 3 accelerometers should be measuring the same magnitude and direction
## Compare compare the relative measures to check for discrepancies

fig=plt.figure()
axs=fig.subplots(2,1,sharex=True)

## use small angle approximation: sin(q) ~ q ~ |a x b|^2 / |a|^2 |b|^2 
q8IMU=axb2(matIMU/np.sum(matIMU**2,axis=0),mat8/np.sum(mat8**2,axis=0),sumall=False)
q816=axb2(matIMU/np.sum(matIMU**2,axis=0),mat16/np.sum(mat16**2,axis=0),sumall=False)
q16IMU=axb2(mat16/np.sum(mat16**2,axis=0),mat8/np.sum(mat8**2,axis=0),sumall=False)

axs[0].set_ylabel("Angle deviation (Deg)")
axs[0].plot(ts,(180/pi)*q8IMU)
axs[0].plot(ts,(180/pi)*q816)
axs[0].plot(ts,(180/pi)*q16IMU)

## Plot the magnitude differences
axs[1].set_ylabel("Relative magnitude")
axs[1].set_xlabel("time (s)")
w8IMU=np.sum(matIMU**2,axis=0)/np.sum(mat8**2,axis=0)
w816=np.sum(mat16**2,axis=0)/np.sum(mat8**2,axis=0)
wIMU16=np.sum(mat16**2,axis=0)/np.sum(matIMU**2,axis=0)
axs[1].plot(ts,w8IMU)
axs[1].plot(ts,w816)
axs[1].plot(ts,wIMU16)


In [ ]:
### Select the stable periods only periods
intervals=[\
          (0,9),\
          (11,20),\
          (23,33),\
          (36.5,45.5),\
          (47,55),\
          (59,70),\
          ]          
t_select,acc_select=idx_filter(ts,mat8,intervals)
plt.plot(acc_select[0,:])
plt.plot(acc_select[1,:])
plt.plot(acc_select[2,:])

In [ ]:
def cal_matrix(params):
    return R(params[3:6]).dot(np.diag(params[0:3]))

def MeanSqError(a,b):
    return 3*np.mean(( a-b) **2 )

In [ ]:
func=MeanSqErrorFunc(mat8,matIMU)

In [ ]:
### Calibration doesnt seem to work??

## Calibrate with rotation and scaling simultaneously
scale=[1,1,1]
lie_vec=np.array([0.00,0,0])
#calibration_matrix=np.diag(scale).dot(R(lie_vec))
#print(calibration_matrix)
func=calibration_cost(mat8,matIMU)

params=minimize(func,[1,1,1,0,0,0])
scale0=params.x[0:3]
lie_vec0=params.x[3:6]

matIMU_cal=cal_matrix(params.x).dot(matIMU)
print(cal_matrix(params.x))


### Initial input
w8IMU=np.sum(matIMU**2,axis=0)/np.sum(mat8**2,axis=0)
q8IMU=axb2(matIMU/np.sum(matIMU**2,axis=0),mat8/np.sum(mat8**2,axis=0),sumall=False)

### corrected input
w8IMU0=np.sum(matIMU_cal**2,axis=0)/np.sum(mat8**2,axis=0)
q8IMU0=axb2(matIMU_cal/np.sum(matIMU**2,axis=0),mat8/np.sum(mat8**2,axis=0),sumall=False)


plt.subplot(2,1,1)
plt.plot(ts,(180/pi)*q8IMU)
plt.plot(ts,(180/pi)*q8IMU0)

plt.subplot(2,1,2)
plt.plot(ts,w8IMU)
plt.plot(ts,w8IMU0)